In [1]:
from Controller import TrainingController, ScenarioCfController
from Parameters import TrainingParameters
from Utils.SaveUtils import load_parameters
from Utils.PrintUtils import print_big
import tensorflow as tf
import json
from dice import EventLogDiCE
import numpy as np
import pandas as pd
import time
from Models import DiCEBinaryDefferentiable
from itertools import chain
from IPython.core.display import display, HTML
from Utils.DiCEHelpers import generate_fake_df, get_trace_with_id, get_longest_trace_row, remove_trail_steps, print_model_prediction_result, remove_tags_for_seq

In [2]:
folder_path = "./SavedModels/%s" % (
    "0.8264_BPI2012WithResource_BaselineLSTMWithResource_2021-06-18 06:11:10.009443" #AOW
)

In [3]:
### Initialise controllers
parameters_json = load_parameters(folder_path=folder_path)
parameters = TrainingParameters(**parameters_json)
tf.random.set_seed(parameters.dataset_split_seed)
np.random.seed(parameters.dataset_split_seed)
parameters.load_model_folder_path = folder_path
trainer = TrainingController(parameters = parameters)
trainer.show_model_info()


| Running on /job:localhost/replica:0/task:0/device:CPU:0  

| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012_with_resource/AOW 

| Model loaded successfully from: ./SavedModels/0.8264_BPI2012WithResource_BaselineLSTMWithResource_2021-06-18 06:11:10.009443  
Model: "baseline_lstm_with_resource"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  2304      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
_________________________________________________________________
lstm_2 (

In [4]:
len(trainer.dataset.df)

13087

In [5]:
########### Get example data from trainer ###########
ordered_test_idx = (list(trainer.test_dataset.unbatch().as_numpy_iterator()))
ordered_test_idx.sort()
print_big(len(ordered_test_idx), "Test set length")


====================Test set length====================
| 1309 


In [6]:
########## Get longest delcined trace for testing ##########
declined_df = get_trace_with_id(trainer.dataset.df.iloc[ordered_test_idx], trainer.model.vocab.vocab_to_index('A_DECLINED_COMPLETE'))
longest_declined_trace_row = get_longest_trace_row(declined_df)
longest_declined_trace_row

,trace,trace_vocab,caseid,amount,resource,resource_orig
8464,"[2, 12, 9, 10, 20, 22, 3, 17, 8, 15, 19, 22, 1...","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",200028,5800.0,"[70, 53, 53, 15, 15, 48, 37, 37, 37, 37, 37, 3...","[<SOS>, 112, 112, 10863, 10863, 11169, 11003, ..."


In [7]:
_, example_activities, _, example_resources, example_amount, _ = trainer.dataset.collate_fn([52])
print("========================================Trace========================================")
print(trainer.model.vocab.list_of_index_to_vocab_2d(example_activities))

========================================Trace========================================
[['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'A_ACCEPTED_COMPLETE', 'A_FINALIZED_COMPLETE', 'O_SELECTED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'O_SENT_BACK_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'O_ACCEPTED_COMPLETE', 'A_APPROVED_COMPLETE', 'A_REGISTERED_COMPLETE', 'A_ACTIVATED_COMPLETE', 'W_Valideren aanvraag_COMPLETE']]


In [8]:
tail_length_to_remove = 15
example_idx_activities, example_idx_resources = remove_trail_steps(example_activities, example_resources, tail_length_to_remove)
example_vocab_activities = trainer.model.vocab.list_of_index_to_vocab_2d(example_idx_activities)[0]
example_vocab_resources = [trainer.model.resources[i] for i in example_idx_resources[0]]
print("========================================Trace after substraction========================================")
print(example_vocab_activities)

========================================Trace after substraction========================================
['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE']


In [9]:
##### Get model output
example_activities_input = tf.constant(example_idx_activities)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

predicted_df = print_model_prediction_result(trainer.model, example_activities_input, example_resources_input, example_amount_input)


========================================Predict result========================================
| Predicted activity with highest probability (0.84) is "<EOS>" 



<PAD>                                      1.505910e-04
<EOS>                                      8.435646e-01
<SOS>                                      3.429898e-05
A_ACCEPTED_COMPLETE                        5.152027e-02
A_ACTIVATED_COMPLETE                       1.720437e-04
A_APPROVED_COMPLETE                        1.530565e-05
A_CANCELLED_COMPLETE                       1.616811e-02
A_DECLINED_COMPLETE                        4.583856e-03
A_FINALIZED_COMPLETE                       6.135483e-03
A_PARTLYSUBMITTED_COMPLETE                 4.327222e-05
A_PREACCEPTED_COMPLETE                     5.293845e-04
A_REGISTERED_COMPLETE                      3.451939e-05
A_SUBMITTED_COMPLETE                       1.233659e-04
O_ACCEPTED_COMPLETE                        2.075346e-05
O_CANCELLED_COMPLETE                       2.228595e

In [10]:
predicted_df

,<PAD>,<EOS>,<SOS>,A_ACCEPTED_COMPLETE,A_ACTIVATED_COMPLETE,A_APPROVED_COMPLETE,A_CANCELLED_COMPLETE,A_DECLINED_COMPLETE,A_FINALIZED_COMPLETE,A_PARTLYSUBMITTED_COMPLETE,...,O_DECLINED_COMPLETE,O_SELECTED_COMPLETE,O_SENT_BACK_COMPLETE,O_SENT_COMPLETE,W_Afhandelen leads_COMPLETE,W_Beoordelen fraude_COMPLETE,W_Completeren aanvraag_COMPLETE,W_Nabellen incomplete dossiers_COMPLETE,W_Nabellen offertes_COMPLETE,W_Valideren aanvraag_COMPLETE
0,0.000151,0.843565,0.000034,0.05152,0.000172,0.000015,0.016168,0.004584,0.006135,0.000043,...,0.000036,0.002354,0.00014,0.000345,5.144636e-07,0.000056,0.073085,0.000178,0.000203,0.000332


In [11]:
no_need_tags = ['<EOS>', '<SOS>', '<PAD>']

possible_activities = [ a for a in list(trainer.model.vocab.vocabs) if  not a in no_need_tags]
possible_resources =  [ r for r in list(trainer.model.resources) if  not r in no_need_tags]

example_vocab_activities_no_tag = remove_tags_for_seq(example_vocab_activities, no_need_tags)

example_vocab_resources_no_tag = remove_tags_for_seq(example_vocab_resources, no_need_tags)

# transform to possible dim
example_idx_activities_no_tag = [ possible_activities.index(v)   for v in example_vocab_activities_no_tag ]

example_idx_resources_no_tag = [
possible_resources.index(v)   for v in example_vocab_resources_no_tag    
]

example_trace_len_no_tag = len(example_vocab_activities_no_tag)
#### Determine feature names for DiCE ####
activity_feature_names = np.array(["activity_step_%d" % (i+1) for i in range(example_trace_len_no_tag)])
resource_feature_names = np.array(["resource_step_%d" % (i+1) for i in range(example_trace_len_no_tag)])
possbile_amount = [min(trainer.dataset.df["amount"]), max(trainer.dataset.df["amount"])]

print_big(example_vocab_activities_no_tag ,title="Example activities without tags", num_marks = 30)
print_big(example_vocab_resources_no_tag ,title="Example resources without tags", num_marks = 30)
print_big(example_amount, title="=================Amount=================", num_marks=16)


==============================Example activities without tags==============================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

==============================Example resources without tags==============================
| ['112', '112', '112'] 

=================================Amount=================================
| [15500.0] 


In [12]:
############ Setting up desired activity ############
# desired_activity = 'A_DECLINED_COMPLETE' # A_DECLINED_COMPLETE, A_APPROVED_COMPLETE
# print_big(desired_activity, "Desired activity")

dice_binary_model = DiCEBinaryDefferentiable(
    model=trainer.model,
    vocab=trainer.model.vocab,
    resources= trainer.model.resources,
#     desired=trainer.model.vocab.vocab_to_index(desired_activity),
    trace_length = example_trace_len_no_tag,
    sos_idx_activity=trainer.model.vocab.vocab_to_index("<SOS>"),
    sos_idx_resource= trainer.model.resources.index('<SOS>'),
    amount_min = possbile_amount[0],
    amount_max = possbile_amount[1],
    possible_resources=possible_resources,
    possible_activities=possible_activities 
)

In [13]:
# fake_df = generate_fake_df(5000, activity_feature_names, resource_feature_names, possible_activities, possible_resources, possbile_amount, example_trace_len_no_tag)

In [14]:
scenario_folder_path = "./SavedModels/%s" % (
#     "0.9860_BPI2012WithResource_BaselineLSTMWithResource_2021-06-25 21:03:20.326179"
    "BPI2012WithResource_ValidPath_2021-06-27 05:38:17.057241"
)
scenario_parameter_json = load_parameters(folder_path=scenario_folder_path)
scenario_parameter = TrainingParameters(**scenario_parameter_json)
scenario_parameter.load_model_folder_path = scenario_folder_path
scenarioController = ScenarioCfController(parameters = scenario_parameter)
scenarioController.show_model_info()


| Running on /job:localhost/replica:0/task:0/device:CPU:0  

| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012_valid_trace/All 

| Model loaded successfully from: ./SavedModels/BPI2012WithResource_ValidPath_2021-06-27 05:38:17.057241  
Model: "scenario_cf_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  832       
_________________________________________________________________
embedding_3 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm_4 (LSTM)                multiple                  8320      
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8320      
_________________________________________________________________
lstm_6 (LSTM)                multiple     

In [15]:
dice = EventLogDiCE(scenarioController.model.activity_vocab, scenarioController.model.resource_vocab, possbile_amount, possible_activities, possible_resources, dice_binary_model, scenarioController.model)

In [16]:
########### Successfully found parameters ###########
#   example_amount_input.numpy(),
#     example_idx_activities_no_tag,
#     example_idx_resources_no_tag,
#     desired_vocab = "A_DECLINED_COMPLETE",
    
#     ## Weight
#     class_loss_weight = 1,
#     scenario_weight=20,
#     distance_loss_weight=0,
#     cat_loss_weight =1,
    
#     ## Training parameters
#     max_iter=200,
#     lr=0.8,
    
#     ## Options
#     use_valid_cf_only=False,
#     use_sampling=True,
#     class_using_hinge_loss=False,
#     scenario_using_hinge_loss=False,
#     use_clipping=True, 

In [17]:
cf_out = dice.run_pls(
    ## Input
    example_amount_input.numpy(),
    example_idx_activities_no_tag,
    example_idx_resources_no_tag,
    desired_vocab = "A_DECLINED_COMPLETE",
    
    ## Weight
    class_loss_weight = 1,
    scenario_weight = 200,
    distance_loss_weight = 1e-8,
    cat_loss_weight = 1e-3,
    
    ## Training parameters
    scenario_threshold = 0.5,
    max_iter=200,
    lr=0.05,
    
    ## Options
    use_valid_cf_only=False,
    use_sampling=True,
    class_using_hinge_loss=False,
    scenario_using_hinge_loss=False,
    use_clipping=True, 
)


====================Model Prediction====================
| Prediction: [<EOS>(1)] | Desired: [A_DECLINED_COMPLETE(7)] 

====================Counterfactual Process====================
| [0] ==========> [1] 

====================!Counterfactual Found in step [19]!====================
| Running time: 2.05 

====================Input Amount====================
| [15500.] 

====================Input Activities====================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

====================Input Resource====================
| ['112', '112', '112'] 

====================Valid CF Amount====================
| 15499.054 

====================Valid CF Activities====================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

====================Valid CF Resource====================
| ['112', 'UNKNOWN', '112'] 

====================Valid CF scenario output====================
| [0.7 1.  0.9 1. ] 


In [18]:
def get_trace_and_prediction_from_model_input(scenarioController, model_input):
    activity, resource, amount = model_input
    
    if len(activity.shape) == 2:
        activity = tf.one_hot(activity, depth= len(scenarioController.model.activity_vocab))
        resource = tf.one_hot(resource, depth= len(scenarioController.model.resource_vocab))
    
    idx_activity = tf.argmax(activity, axis=-1).numpy().tolist()[0]
    vocab_activity = scenarioController.model.activity_vocab.list_of_index_to_vocab(idx_activity)
    print_big(vocab_activity, "Activitiy")
    
    idx_resource = tf.argmax(resource, axis=-1).numpy().tolist()[0]
    vocab_resource = scenarioController.model.resource_vocab.list_of_index_to_vocab(idx_resource)
    print_big(vocab_resource, "Resource")
    
    sigmoid_output = tf.nn.sigmoid(scenarioController.model(*model_input, training=False)[0]).numpy()
    print_big(np.around(sigmoid_output.flatten(), decimals=3), "Prediction from scenario")
    
## ! The cf_out should be the same as temp_model_input

In [19]:
print_big("Original input")
get_trace_and_prediction_from_model_input(scenarioController, [example_activities_input, example_resources_input, example_amount_input])
print_big("Input CF")
get_trace_and_prediction_from_model_input(scenarioController, dice.model_input)
print_big("Updated CF")
get_trace_and_prediction_from_model_input(scenarioController, dice.temp_model_input)


| Original input 

====================Activitiy====================
| ['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

====================Resource====================
| ['<SOS>', '112', '112', '112'] 

====================Prediction from scenario====================
| [0.695 1.    1.    1.   ] 

| Input CF 

====================Activitiy====================
| ['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

====================Resource====================
| ['<SOS>', '112', '112', '112'] 

====================Prediction from scenario====================
| [0.695 1.    1.    1.   ] 

| Updated CF 

====================Activitiy====================
| ['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

====================Resource====================
| ['<SOS>', '112', 'UNKNOWN', '112'] 

====================Prediction from scenario====================
| [0.695 

In [20]:
def get_scenario_prediction_from_vocabs(scenarioController, cf_out):
    cf_amount, cf_activity, cf_resource = cf_out

    input_activity = [scenarioController.model.activity_vocab.sos_idx()] + scenarioController.model.activity_vocab.list_of_vocab_to_index(cf_activity)
    input_activity = tf.constant([input_activity]) 

    input_resource = [scenarioController.model.resource_vocab.sos_idx()] + scenarioController.model.resource_vocab.list_of_vocab_to_index(cf_resource)
    input_resource = tf.constant([input_resource])

    input_amount = tf.constant([cf_amount])
    
    print_big(scenarioController.model(input_activity, input_resource, input_amount)[0].numpy().flatten().tolist(), "Scenario CF Output")

In [21]:
get_scenario_prediction_from_vocabs(scenarioController, cf_out)


====================Scenario CF Output====================
| [0.823211669921875, 8.287096977233887, 2.646289825439453, 10.151716232299805] 


In [22]:
class GoodIsGood(object):
    pass

In [23]:
g = GoodIsGood()

In [24]:
g.__name__

AttributeError: 'GoodIsGood' object has no attribute '__name__'